In [27]:
import pyreadr
import math
import numpy as np
from scipy.optimize import minimize

In [88]:
def find_optimal_strategy_1(rho, sigma, beta, _lambda):
    w = np.asarray([1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12])
    def strategy_1(w):
        return(-1 * float(rho.T @ w - _lambda * math.sqrt(w.T @ sigma @ w)))
    constraints = ({"type": "eq", "fun": lambda w: 1-sum(w)},
                {"type": "ineq", "fun": lambda w: beta.T@w+0.5},
                {"type": "ineq", "fun": lambda w: -beta.T@w+0.5},
                {"type": "ineq", "fun": lambda w: 2+min(w)},
                {"type": "ineq", "fun": lambda w: 2-max(w)})
    minim = minimize(strategy_1, w, constraints = constraints)
    return(minim.x)

result = pyreadr.read_r('/Users/Ravi/Desktop/FE 630/FE-630-Final-Project/strategy_params.RData')
rho = np.asarray(result['Expected_Return'])
sigma = np.asarray(result['Covariance_Matrix'])
beta = np.asarray(result['Beta'])
_lambda = 0.1
optimal_strat1 = find_optimal_strategy_1(rho, sigma, beta, _lambda)

# Optimal Weights
print(optimal_strat1)

# Return of Optimal Weights
print(float(rho.T @ optimal_strat1))

# Variance of Optimal Weights
print(float(optimal_strat1.T @ sigma @ optimal_strat1))

[-2.00016375 -1.99949428  1.99984351  0.59731148  1.9999255   2.
  0.02055636 -1.99973881 -0.21567121  1.99861974  0.59897383 -2.00016235]
0.5777405879492608
4.101019195232277
